# Contrastive Learning From Scratch - DistilBERT

An attempt to build contrastive learning model from scratch. Parts include:

- Loading and preparing Wiki-1M data for model input
- Contrastive learning model
  - Forward passing using pre-trained model
  - Constrastive layer
  - Calculate loss
- Training procedure
  - Default trainer optimizer
  - Default trainer hyper-parameters

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!cp "/content/gdrive/MyDrive/proposal/valerie/distil_face.zip" .
!unzip -q distil_face.zip

In [ ]:
% cd distil_face

/content/distil_face


In [ ]:
! pip install datasets

In [ ]:
! pip install -r requirements.txt

In [ ]:
% cd SentEval

/content/distil_face/SentEval


In [ ]:
! python setup.py install

running install
running bdist_egg
running egg_info
writing SentEval.egg-info/PKG-INFO
writing dependency_links to SentEval.egg-info/dependency_links.txt
writing top-level names to SentEval.egg-info/top_level.txt
reading manifest file 'SentEval.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'SentEval.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/senteval
copying build/lib/senteval/trec.py -> build/bdist.linux-x86_64/egg/senteval
copying build/lib/senteval/sick.py -> build/bdist.linux-x86_64/egg/senteval
copying build/lib/senteval/probing.py -> build/bdist.linux-x86_64/egg/senteval
copying build/lib/senteval/sst.py -> build/bdist.linux-x86_64/egg/senteval
copying build/lib/senteval/snli.py -> build/bdist.linux-x86_64/egg/senteval
copying build/lib/senteval/mrpc.py -> build/bdist.linux-x86_64/egg/senteval
copying build/

In [ ]:
# ! pip freeze

absl-py==1.0.0
aiohttp==3.8.1
aiosignal==1.2.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.11.4
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
async-timeout==4.0.2
asynctest==0.13.0
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.4.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==4.1.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.4
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.10
cached-property==1.5.2
cachetools==4.2.4
catalogue==1.0.0
certifi==2021.10.8
cffi==1.15.0
cftime==1.6.0
chardet==3.0.4
charset-normalizer==2.0.12
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cupy-cuda111==9.4.0
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.28
daft==0.0.4
dask==2.12.0
datascience==0.10.6
d

In [ ]:
% cd ..

/content/distil_face


In [ ]:
# import os

# # Set Project home
# PROJECT_HOME = os.path.join('/',
#                             'workspace',
#                             'gatech',
#                             'cs7643-deep-learning',
#                             'contrastive-learning-in-distilled-models')
# %cd {PROJECT_HOME}

# # Load project code
# %reload_ext autoreload
# %autoreload 2

# import sys
# sys.path.insert(0, './src')

# import distilface


import src.distilface

In [ ]:
! bash ./scripts/download_training_data.sh
! bash ./scripts/download_eval_data.sh

--2022-04-15 14:07:50--  https://huggingface.co/datasets/princeton-nlp/datasets-for-simcse/resolve/main/wiki1m_for_simcse.txt
Resolving huggingface.co (huggingface.co)... 34.224.55.150, 34.200.173.213, 34.198.1.82, ...
Connecting to huggingface.co (huggingface.co)|34.224.55.150|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/princeton-nlp/datasets-for-simcse/7b1825863a99aa76479b0456f7c210539dfaeeb69598b41fb4de4f524dd5a706 [following]
--2022-04-15 14:07:50--  https://cdn-lfs.huggingface.co/datasets/princeton-nlp/datasets-for-simcse/7b1825863a99aa76479b0456f7c210539dfaeeb69598b41fb4de4f524dd5a706
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 108.159.227.123, 108.159.227.69, 108.159.227.71, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|108.159.227.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120038621 (114M) [text/plain]
Saving to: ‘./data/training/wik

## 1. Loading and Preparing Wiki-1M data

Use huggingface `datasets` library to load local file data.

In [ ]:
import numpy as np

from datasets import load_dataset

# data_files = {'train': train_name, 'validation': valid_name}
data_files = {'train': 'data/training/wiki1m_for_simcse.txt'}
datasets = load_dataset('text', data_files=data_files, cache_dir='./data/')
# datasets = load_dataset('text', data_files=data_files)

Using custom data configuration default-053c82cb395e5e49
Reusing dataset text (./data/text/default-053c82cb395e5e49/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Unsupervised / Self-supervised dataset

column_names = datasets["train"].column_names
sent0_cname = column_names[0]
sent1_cname = column_names[0]

print('column_names:', column_names)
print('sent0_cname:', sent0_cname, '| sent1_cname:', sent1_cname)

column_names: ['text']
sent0_cname: text | sent1_cname: text


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
def prepare_features(examples):
    total = len(examples[sent0_cname])

    # Avoid "None" fields 
    for idx in range(total):
        if examples[sent0_cname][idx] is None:
            examples[sent0_cname][idx] = " "
        if examples[sent1_cname][idx] is None:
            examples[sent1_cname][idx] = " "
    
    sentences = examples[sent0_cname] + examples[sent1_cname]

    sent_features = tokenizer(
        sentences,
        max_length=32,
        truncation=True,
        padding=True,
    )

    features = {}
    for key in sent_features:
        features[key] = [[sent_features[key][i], sent_features[key][i+total]] for i in range(total)]

    return features

In [ ]:
train_dataset = datasets["train"].map(prepare_features,
                                      batched=True,
                                    #   num_proc=24,
                                      remove_columns=column_names)
# validation_dataset = datasets["validation"].map(prepare_features,
#                                       batched=True,
#                                     #   num_proc=24,
#                                       remove_columns=column_names)

Loading cached processed dataset at ./data/text/default-053c82cb395e5e49/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-5340c05a8327483a.arrow


In [ ]:
train_dataset.num_rows

1000000

In [ ]:
str(train_dataset['input_ids'][0][0])

'[101, 26866, 1999, 2148, 2660, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]'

In [ ]:
str(train_dataset['input_ids'][0][1])

'[101, 26866, 1999, 2148, 2660, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]'

In [ ]:
train_dataset.features.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
% cd src

/content/distil_face/src


Sentence 1 and Sentence 2 are the same sentence

## 2. Contrastive Learning Model

In [ ]:
import torch
import torch.nn as nn

from transformers import AutoTokenizer, DistilBertModel, DistilBertPreTrainedModel, AutoConfig
from transformers.modeling_outputs import SequenceClassifierOutput, BaseModelOutputWithPooling

from distilface.modules.pooler import Pooler
from distilface.modules.similarity import Similarity


class DistilBertCLModel(DistilBertPreTrainedModel):
    def __init__(self, config, pooler_type='avg_last4', temp=0.05):
        super().__init__(config)

        self.config = config
        self.pooler_type = pooler_type
        self.temp = 0.05

        self.distilbert = DistilBertModel(config)
        self.pooler = Pooler(pooler_type)
        self.sim = Similarity(temp=temp)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None):
        if self.training:
            return self.cl_forward(self.distilbert, input_ids, attention_mask)
        else:
            return self.sent_emb(self.distilbert, input_ids, attention_mask)

    def cl_forward(self, encoder, input_ids=None, attention_mask=None):
        batch_size = input_ids.size(0)
        num_sent = input_ids.size(1)  # Number of sentences in one instance: 2 sentences

        # Flatten all input tensors
        input_ids = input_ids.view((-1, input_ids.size(-1))) # (bs * num_sent, len)
        attention_mask = attention_mask.view((-1, attention_mask.size(-1))) # (bs * num_sent len)

        # Pre-trained Model Encoder
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )

        # Pooling
        pooler_output = self.pooler(attention_mask, outputs)
        pooler_output = pooler_output.view((batch_size, num_sent, pooler_output.size(-1)))  # (bs, num_sent, hidden)

        # Separate representation
        z1, z2 = pooler_output[:, 0], pooler_output[:, 1]

        # Cosine similarity
        cos_sim = self.sim(z1.unsqueeze(1), z2.unsqueeze(0))

        # Calculate contrastive loss
        criterion = nn.CrossEntropyLoss()
        labels = torch.arange(cos_sim.size(0)).long().to(self.device)
        loss = criterion(cos_sim, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=cos_sim,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def sent_emb(self, encoder, input_ids=None, attention_mask=None):
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )
        pooler_output = self.pooler(attention_mask, outputs)

        return BaseModelOutputWithPooling(
            pooler_output=pooler_output,
            last_hidden_state=outputs.last_hidden_state,
            hidden_states=outputs.hidden_states,
        )


pretrained_model_name = 'distilbert-base-uncased'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained(pretrained_model_name)

model = DistilBertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

model.eval();


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertCLModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertCLModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertCLModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 2.1 Initial DistilBERT embeddings performance

In [ ]:
% cd ../SentEval

/content/distil_face/SentEval


In [ ]:
import senteval

In [ ]:
% cd ..

/content/distil_face


In [ ]:



def prepare(params, samples):
    return

def batcher(params, batch):
    sentences = [" ".join(s) for s in batch]
    batch = tokenizer.batch_encode_plus(
        sentences,
        return_tensors="pt",
        padding=True,
    )

    for k in batch:
        batch[k] = batch[k].to(device)

    with torch.no_grad():
        outputs = model(**batch)

    pooled_result = outputs.pooler_output.cpu()

    return pooled_result


def evaluate_model():
    PATH_TO_DATA = "./data"

    params = {"task_path": PATH_TO_DATA, "usepytorch": True, "kfold": 10}
    tasks = ["STSBenchmark", 'STS12', 'STS13', 'STS14', 'STS15']

    se = senteval.engine.SE(params, batcher, prepare)
    results = se.eval(tasks)
    print('results: ', results)
    print('STS12: ', results["STS12"]["all"]["spearman"]["all"])
    print('STS13: ', results["STS13"]["all"]["spearman"]["all"])
    print('STS14: ', results["STS14"]["all"]["spearman"]["all"])
    print('STS15: ', results["STS15"]["all"]["spearman"]["all"])
    print('STSB: ', results["STSBenchmark"]["test"]["spearman"][0])

    return results

In [ ]:
results = evaluate_model()
results

/content/distil_face/SentEval/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
/content/distil_face/SentEval/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


results:  {'STSBenchmark': {'train': {'pearson': (0.6070898763008297, 0.0), 'spearman': SpearmanrResult(correlation=0.5955309594381452, pvalue=0.0), 'nsamples': 5749}, 'dev': {'pearson': (0.6714695311578094, 3.1816154363896114e-197), 'spearman': SpearmanrResult(correlation=0.6842821404252671, pvalue=1.0922682924251785e-207), 'nsamples': 1500}, 'test': {'pearson': (0.5632637995447413, 2.9049540160634865e-116), 'spearman': SpearmanrResult(correlation=0.590522771009074, pvalue=2.1926696952431418e-130), 'nsamples': 1379}, 'all': {'pearson': {'all': 0.6130842261690994, 'mean': 0.6139410690011268, 'wmean': 0.6112777903526173}, 'spearman': {'all': 0.6146095387318827, 'mean': 0.6234452902908288, 'wmean': 0.6101601295397903}}}, 'STS12': {'MSRpar': {'pearson': (0.42118860126558844, 1.3131983706650327e-33), 'spearman': SpearmanrResult(correlation=0.45987913574306377, pvalue=1.6077021457625966e-40), 'nsamples': 750}, 'MSRvid': {'pearson': (0.6353183836264454, 5.058902811436889e-86), 'spearman': Sp

{'STS12': {'MSRpar': {'nsamples': 750,
   'pearson': (0.42118860126558844, 1.3131983706650327e-33),
   'spearman': SpearmanrResult(correlation=0.45987913574306377, pvalue=1.6077021457625966e-40)},
  'MSRvid': {'nsamples': 750,
   'pearson': (0.6353183836264454, 5.058902811436889e-86),
   'spearman': SpearmanrResult(correlation=0.6401648261692996, pvalue=1.0195217279722163e-87)},
  'SMTeuroparl': {'nsamples': 459,
   'pearson': (0.5022992872170757, 1.0330490467720212e-30),
   'spearman': SpearmanrResult(correlation=0.5931239653520929, pvalue=5.929140221048737e-45)},
  'all': {'pearson': {'all': 0.45148177281626845,
    'mean': 0.5741005705164426,
    'wmean': 0.5751895779483213},
   'spearman': {'all': 0.4759552744453918,
    'mean': 0.5871739677651739,
    'wmean': 0.590629584233438}},
  'surprise.OnWN': {'nsamples': 750,
   'pearson': (0.6877112853360426, 3.984348473186997e-106),
   'spearman': SpearmanrResult(correlation=0.6910592978736139, pvalue=1.4770691983640824e-107)},
  'surpri

## 3. Trainer

In [ ]:
# import mlflow

from transformers import Trainer, TrainingArguments
from transformers import default_data_collator
lr =  1e-5  ## ,5e-05 3e-5 #

# training_args = TrainingArguments(
#     output_dir='output',
#     overwrite_output_dir=True,
#     learning_rate=5e-05,
#     weight_decay=0.0,
#     num_train_epochs=1,
#     eval_steps=100,
# )
training_args = TrainingArguments(
    output_dir='output',
    overwrite_output_dir=True,
    learning_rate=lr,
    weight_decay=0.0,
    num_train_epochs=1,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    # max_steps=2,
    # save_steps=2,
    # logging_steps=2
    max_steps=30000,
    save_steps=5000,
    logging_steps=30000,
    fp16=True
)

model.train()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

max_steps is given, it will override any value given in num_train_epochs
Using amp half precision backend


In [ ]:
train_result = trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1000000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss
30000,0.000000


Saving model checkpoint to output/checkpoint-30000
Configuration saved in output/checkpoint-30000/config.json
Model weights saved in output/checkpoint-30000/pytorch_model.bin
tokenizer config file saved in output/checkpoint-30000/tokenizer_config.json
Special tokens file saved in output/checkpoint-30000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




In [ ]:
train_result = trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1000000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss
30000,0.000000


Saving model checkpoint to output/checkpoint-30000
Configuration saved in output/checkpoint-30000/config.json
Model weights saved in output/checkpoint-30000/pytorch_model.bin
tokenizer config file saved in output/checkpoint-30000/tokenizer_config.json
Special tokens file saved in output/checkpoint-30000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




In [ ]:
train_result = trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1000000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss


Step,Training Loss
30000,0.000000


Saving model checkpoint to output/checkpoint-30000
Configuration saved in output/checkpoint-30000/config.json
Model weights saved in output/checkpoint-30000/pytorch_model.bin
tokenizer config file saved in output/checkpoint-30000/tokenizer_config.json
Special tokens file saved in output/checkpoint-30000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




In [ ]:
train_result = trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1000000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss


## 4. Evaluate DistilBert CL Model performance

In [ ]:
model.eval()

results = evaluate_model()
# 5e-05
results

/content/distil_face/SentEval/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
/content/distil_face/SentEval/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


results:  {'STSBenchmark': {'train': {'pearson': (0.6413343191631499, 0.0), 'spearman': SpearmanrResult(correlation=0.622760474239443, pvalue=0.0), 'nsamples': 5749}, 'dev': {'pearson': (0.6953448953308116, 3.6176611774898064e-217), 'spearman': SpearmanrResult(correlation=0.7070068117563632, pvalue=1.215876377436612e-227), 'nsamples': 1500}, 'test': {'pearson': (0.6041644004635148, 5.669138920231195e-138), 'spearman': SpearmanrResult(correlation=0.6193071799582047, pvalue=7.741720799461172e-147), 'nsamples': 1379}, 'all': {'pearson': {'all': 0.6465638650460731, 'mean': 0.646947871652492, 'wmean': 0.6447833857330034}, 'spearman': {'all': 0.642011939300257, 'mean': 0.649691488651337, 'wmean': 0.6368549820583527}}}, 'STS12': {'MSRpar': {'pearson': (0.42446125666001966, 3.6951898351027358e-34), 'spearman': SpearmanrResult(correlation=0.46275007748686425, pvalue=4.5392655293692836e-41), 'nsamples': 750}, 'MSRvid': {'pearson': (0.6606497054808091, 3.094217485925751e-95), 'spearman': Spearman

{'STS12': {'MSRpar': {'nsamples': 750,
   'pearson': (0.42446125666001966, 3.6951898351027358e-34),
   'spearman': SpearmanrResult(correlation=0.46275007748686425, pvalue=4.5392655293692836e-41)},
  'MSRvid': {'nsamples': 750,
   'pearson': (0.6606497054808091, 3.094217485925751e-95),
   'spearman': SpearmanrResult(correlation=0.6666661738830504, pvalue=1.4739747238522755e-97)},
  'SMTeuroparl': {'nsamples': 459,
   'pearson': (0.5157429394004893, 1.477523219245375e-32),
   'spearman': SpearmanrResult(correlation=0.605070807801308, pvalue=3.534959915927703e-47)},
  'all': {'pearson': {'all': 0.49386073468421443,
    'mean': 0.586864325617115,
    'wmean': 0.5875849404404245},
   'spearman': {'all': 0.502326183517632,
    'mean': 0.5997693994613249,
    'wmean': 0.6024954570812042}},
  'surprise.OnWN': {'nsamples': 750,
   'pearson': (0.6940203453197574, 7.71665955509072e-109),
   'spearman': SpearmanrResult(correlation=0.6931241481029086, pvalue=1.8926797886685526e-108)},
  'surprise.S

In [ ]:
model.eval()

results = evaluate_model()
# 3e-05
results

/content/distil_face/SentEval/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
/content/distil_face/SentEval/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


results:  {'STSBenchmark': {'train': {'pearson': (0.7438214017408256, 0.0), 'spearman': SpearmanrResult(correlation=0.7153129840027083, pvalue=0.0), 'nsamples': 5749}, 'dev': {'pearson': (0.7556052735790904, 1.5563634968643622e-277), 'spearman': SpearmanrResult(correlation=0.7612821730557856, pvalue=3.9263252569098916e-284), 'nsamples': 1500}, 'test': {'pearson': (0.7424669070706221, 7.2743909795608805e-242), 'spearman': SpearmanrResult(correlation=0.7345777391009741, pvalue=3.402979372929511e-234), 'nsamples': 1379}, 'all': {'pearson': {'all': 0.7417080050442152, 'mean': 0.747297860796846, 'wmean': 0.7456535713754091}, 'spearman': {'all': 0.7307545522805707, 'mean': 0.737057632053156, 'wmean': 0.7263839020439837}}}, 'STS12': {'MSRpar': {'pearson': (0.5225283625113606, 8.934225383277799e-54), 'spearman': SpearmanrResult(correlation=0.5530207602070452, pvalue=2.682647894800919e-61), 'nsamples': 750}, 'MSRvid': {'pearson': (0.8216925869772372, 8.033335721568975e-185), 'spearman': Spearma

{'STS12': {'MSRpar': {'nsamples': 750,
   'pearson': (0.5225283625113606, 8.934225383277799e-54),
   'spearman': SpearmanrResult(correlation=0.5530207602070452, pvalue=2.682647894800919e-61)},
  'MSRvid': {'nsamples': 750,
   'pearson': (0.8216925869772372, 8.033335721568975e-185),
   'spearman': SpearmanrResult(correlation=0.8184132989400847, pvalue=3.7501524731480714e-182)},
  'SMTeuroparl': {'nsamples': 459,
   'pearson': (0.5053420868954367, 4.015780376211538e-31),
   'spearman': SpearmanrResult(correlation=0.6078585905890408, pvalue=1.036654205326215e-47)},
  'all': {'pearson': {'all': 0.632384955165421,
    'mean': 0.6332524458087977,
    'wmean': 0.6492725162638885},
   'spearman': {'all': 0.5897539328895788,
    'mean': 0.6382890082675989,
    'wmean': 0.6542238026800054}},
  'surprise.OnWN': {'nsamples': 750,
   'pearson': (0.7192543381065248, 1.9665884582502716e-120),
   'spearman': SpearmanrResult(correlation=0.6897325841584372, pvalue=5.479976584694133e-107)},
  'surprise.S

In [ ]:
model.eval()

results = evaluate_model()
# 1e-05
results

/content/distil_face/SentEval/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
/content/distil_face/SentEval/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


results:  {'STSBenchmark': {'train': {'pearson': (0.7624856292575045, 0.0), 'spearman': SpearmanrResult(correlation=0.735292352044148, pvalue=0.0), 'nsamples': 5749}, 'dev': {'pearson': (0.7670603397832837, 4.8495110487467785e-291), 'spearman': SpearmanrResult(correlation=0.7733578219587147, pvalue=8.399089838055714e-299), 'nsamples': 1500}, 'test': {'pearson': (0.7590813642973432, 5.610516917261442e-259), 'spearman': SpearmanrResult(correlation=0.7500014513164092, pvalue=1.8577709076615907e-249), 'nsamples': 1379}, 'all': {'pearson': {'all': 0.7594495208635771, 'mean': 0.7628757777793771, 'wmean': 0.7627368560086178}, 'spearman': {'all': 0.748531465934916, 'mean': 0.752883875106424, 'wmean': 0.7442610646969411}}}, 'STS12': {'MSRpar': {'pearson': (0.5145387013058537, 6.288083753340994e-52), 'spearman': SpearmanrResult(correlation=0.5517000882734269, pvalue=5.896786928747554e-61), 'nsamples': 750}, 'MSRvid': {'pearson': (0.8368854218889684, 6.038410451288584e-198), 'spearman': Spearmanr

{'STS12': {'MSRpar': {'nsamples': 750,
   'pearson': (0.5145387013058537, 6.288083753340994e-52),
   'spearman': SpearmanrResult(correlation=0.5517000882734269, pvalue=5.896786928747554e-61)},
  'MSRvid': {'nsamples': 750,
   'pearson': (0.8368854218889684, 6.038410451288584e-198),
   'spearman': SpearmanrResult(correlation=0.8328335559450619, pvalue=2.5692581602338342e-194)},
  'SMTeuroparl': {'nsamples': 459,
   'pearson': (0.5167512604470813, 1.0662212541579138e-32),
   'spearman': SpearmanrResult(correlation=0.6168089089754981, pvalue=1.8577936743912675e-49)},
  'all': {'pearson': {'all': 0.6620425932915542,
    'mean': 0.6423278412194261,
    'wmean': 0.657582203189099},
   'spearman': {'all': 0.6107695441625319,
    'mean': 0.6477363855800882,
    'wmean': 0.6632476357825673}},
  'surprise.OnWN': {'nsamples': 750,
   'pearson': (0.7320982946175422, 7.739829836414225e-127),
   'spearman': SpearmanrResult(correlation=0.7013097410846821, pvalue=4.613192027287151e-112)},
  'surprise.

In [ ]:
model.eval()

results = evaluate_model()
# 5e-07
results

In [ ]:
! nvidia-smi

Thu Mar 24 11:39:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    29W /  70W |   2958MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------